In [1]:
import requests
import pandas as pd
import datetime
import numpy as np
from bs4 import BeautifulSoup
import argparse
from cassandra.cluster import Cluster

In [3]:
#job_types = ['software%20engineer', 'data%20analyst', 'systems%20engineer']
job_types = [('software_engineer', 'software%20engineer'), ('data_analyst', 'data%20analyst'), 
             ('systems_engineer', 'systems%20engineer')]
job_titles_parsed = []
job_descriptions_parsed = []
job_locations_parsed = []
CV_URLs_parsed = []

#job_type_to_search = job_types[0][1]  # software%20engineer
#job_type_to_search = job_types[1][1]  # data%20analyst
job_type_to_search = job_types[2][1]  # systems%20engineer

# livecareer only provides 20 pages having 10 CV on each even if they show more than 200 CV results
for i in range(20):
    pg_number = i + 1
    print('pg_number:' + str(pg_number))

    if pg_number == 1:
        #https://www.livecareer.com/resume-search/search?jt=software%20engineer
        URL_livecareer = "https://www.livecareer.com/resume-search/search?jt=" + job_type_to_search
    else:
        #https://www.livecareer.com/resume-search/search?jt=software%20engineer&pg=2
        URL_livecareer = "https://www.livecareer.com/resume-search/search?jt=" + job_type_to_search + \
                                                                                "&pg="+ str(pg_number)

    print('URL: ' + URL_livecareer)
    page_livecareer = requests.get(URL_livecareer)

    soup_livecareer = BeautifulSoup(page_livecareer.content, "html.parser")
    results_livecareer = soup_livecareer.find(id="resume-list-outer")

    job_titles = results_livecareer.find_all("p", class_="h3 resume-title")
    job_titles_parsed.extend([title.text.strip() for title in job_titles])
    
    job_descriptions = results_livecareer.find_all("p", class_="description hidden-xs")
    job_descriptions_parsed.extend([description.text.strip() for description in job_descriptions])
    
    job_locations = results_livecareer.find_all("p", class_="location")
    job_locations_parsed.extend([location.find("span", itemprop="contentLocation").text for location in job_locations])
    
    CV_URLs = results_livecareer.find_all("input", class_="infoWindow")
    CV_URLs_parsed.extend([URL["value"].split('|')[1] for URL in CV_URLs])
    
dict_job_data = {'job_titles': job_titles_parsed, 'job_descriptions': job_descriptions_parsed, 
                     'job_locations': job_locations_parsed,'CV_URLs': CV_URLs_parsed}
    
    
df_job_data = pd.DataFrame(dict_job_data)
#print(df_job_data)
print(df_job_data['CV_URLs'])
        

pg_number:1
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer
pg_number:2
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer&pg=2
pg_number:3
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer&pg=3
pg_number:4
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer&pg=4
pg_number:5
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer&pg=5
pg_number:6
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer&pg=6
pg_number:7
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer&pg=7
pg_number:8
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer&pg=8
pg_number:9
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer&pg=9
pg_number:10
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engineer&pg=10
pg_number:11
URL: https://www.livecareer.com/resume-search/search?jt=systems%20engi

In [4]:
cluster = Cluster(['127.0.0.1'])  #cluster = Cluster(ip_address)
session = cluster.connect("jobsearch_keyspace")  #session = cluster.connect(keyspace_name)

# Let's search for software_engineer
#job_type_column = job_types[0][0]  # software_engineer
#job_type_column = job_types[1][0]  # data_analyst
job_type_column = job_types[2][0]  # sysytems_engineer

# !!!! Cassandra doe not have NOT EQUAL or NOT NULL filters so we have to do some workarounds...
select_job_type_skills = "SELECT idx, " + job_type_column + " from job_type"
job_type_skills = pd.DataFrame(session.execute(select_job_type_skills)).sort_values(by=['idx'])[job_type_column].values.tolist()
# removing the NULL/emopty records
job_type_skills = list(filter(None, job_type_skills))

select_maxID = "SELECT max(User_ID) from job_seeker"
maxID = pd.DataFrame(session.execute(select_maxID))

# Prepare the INSERT command
insert_job_seeker = "INSERT INTO job_seeker(User_ID, Job_Title, Skillset) VALUES (?,?,?)"
prepared_job_seeker = session.prepare(insert_job_seeker)

#job_type_skills = session.execute(select_job_type_skills)  # >> returns save cassandra.cluster.ResultSet obj
#job_type_skills = pd.DataFrame(session.execute(select_job_type_skills)).values.tolist() # >> list of lists, 
# ABOVE > each job is saved as a list of a single element (which is skill) in a list named job_type_skills

#print(maxID['system_max_idx'][0]) # >> returns Row obj
print('max_user_id in the table: ' + str(maxID['system_max_user_id'][0]))

print('count_skills: ' + str(len(job_type_skills)))
print('all skills for ' + job_type_column)
print(job_type_skills) 


max_user_id in the table: 601
count_skills: 91
all skills for systems_engineer
['2G', '3Com', '3G', '4G LTE', '5GNR', 'Abend-AID', 'Accounting System', 'Active Directory', 'ADS', 'Aerodynamics', 'Algorithm Design', 'Algorithm Development', 'ANSYS', 'Apache', 'Assembly', 'AWS', 'C#', 'C++', 'CAD', 'Catia', 'CCNA', 'CCNP', 'CICS', 'Cisco', 'Citrix', 'COBOL', 'DB2', 'Disaster Recovery', 'DoDAF', 'EIGRP', 'ENA', 'ERP', 'Firewall', 'Frame-Relay', 'GCP', 'GridPro', 'GSM', 'HP EVA', 'HSPA', 'IBM SAN', 'IBM TS', 'IDMS', 'IEC 60601-1', 'IEC 60601-1-2', 'IEC 60601-2-4', 'IEC 62304', 'IIS', 'ISO 14971', 'JAVA', 'JCL', 'LDAP', 'MATLAB', 'MBSE', 'MVS', 'Network Protocols', 'Object-oriented', 'OpenView', 'OPSPF', 'ORACLE', 'Perl', 'PLC', 'POP3', 'Python', 'Radius', 'RAID', 'RSA SecurID', 'SCADA', 'Shell script', 'SNMP', 'Solidworks', 'SQL', 'SQL Server', 'Storage Area Network', 'Switch', 'Sybase', 'SysML', 'System Admin', 'System Integration', 'TCP/IP', 'TCPIP', 'Tivoli', 'UDP', 'VHDL', 'Visual Basi

In [16]:
# we will create a list of 0, 1 where 0 shows the job_seeker doesn't have the skill and 1 means she has the skill
result_search_skills = np.zeros(len(job_type_skills), int)

start_time = datetime.datetime.now()
print('start time: ' + str(start_time))
for url in df_job_data['CV_URLs']:
    CV_details = requests.get("https://www.livecareer.com" + url)
    maxID = pd.DataFrame(session.execute(select_maxID))

    for i, skill in enumerate(job_type_skills):
        if CV_details.text.lower().find(skill.lower()) != -1:
            #print(str(i) + skill)
            result_search_skills[i] = 1
            
    # we need to convert the nparray to the list for Cassandra
    l_result_search_skills = list(result_search_skills)
    if maxID['system_max_user_id'][0] == None:
        userID = 1
    else:
        userID = maxID['system_max_user_id'][0] + 1

    #User_ID, Job_Title, Skillset
    session.execute(prepared_job_seeker, (userID, job_type_column, l_result_search_skills))
    
end_time = datetime.datetime.now()
print('end time: ' + str(end_time))
print('total calculation time: ' + str(end_time - start_time))
#print(result_search_skills)

start time: 2020-10-23 18:26:14.612124
end time: 2020-10-23 18:27:54.332543
total calculation time: 0:01:39.720419


In [5]:
q_all_job_seekers = "SELECT * from job_seeker"
all_job_seekers = pd.DataFrame(session.execute(q_all_job_seekers)).sort_values(by=['user_id'])
all_job_seekers


,user_id,job_title,skillset
154,1,software_engineer,"[1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ..."
188,2,software_engineer,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
595,3,software_engineer,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
201,4,software_engineer,"[0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
46,5,software_engineer,"[0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, ..."
...,...,...,...
277,597,systems_engineer,"[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, ..."
434,598,systems_engineer,"[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, ..."
164,599,systems_engineer,"[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, ..."
483,600,systems_engineer,"[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, ..."
